In [324]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [325]:
from utils.UpdateDatabase import *

# Data Formatting

In [326]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from urllib.parse import urljoin
from dotenv import load_dotenv
from datetime import *
from bson import ObjectId
import pandas as pd
import numpy as np
import requests
import random
import torch
import json
import os
load_dotenv()

True

In [327]:
from utils.FlattenData import *
from modules.PremoAPI import *
from utils.SafeDataConverters import *

In [328]:
api = PremoAPI()

In [329]:
username = os.getenv("mongodb_user")
password = os.getenv("mongodb_password")

In [330]:
from modules.database import Database
db = Database(username, password)

In [331]:
play_data: list[dict] = api.play_data

In [332]:
flat_out = flatten_output(db.to_json())
flat_in = flatten_input(db.to_json())

In [333]:
inp_df = pd.DataFrame(flat_in).sort_values(by=['order_id', 'sort_order'], ascending=True).convert_dtypes()
inp_df['sort_order'] = inp_df.groupby(['order_id', 'product_id'])['sort_order'].transform(lambda x: x.rank(method='dense').astype(int) - 1)
inp_df = inp_df.map(safe_to_datetime)
inp_df.head()

,order_id,delivery_date,order_created_at,product_id,material,color,task_title,task_duration,sort_order,workspace
3,1,2024-07-13 14:28:51,2024-07-08 14:28:51,67053bafda93609bc1305297,Katoenen draagtas gekleurd,06,Vullen,3600,0,Soweco
2,1,2024-07-13 14:28:51,2024-07-08 14:28:51,67053bafda93609bc1305297,Katoenen draagtas gekleurd,06,Tampon drukken (TB),1440,1,T2 1|2|3
0,1,2024-07-13 14:28:51,2024-07-08 14:28:51,67053bafda93609bc1305297,Katoenen draagtas gekleurd,06,Inpakken,180,2,Inpak
1,1,2024-07-13 14:28:51,2024-07-08 14:28:51,67053bafda93609bc1305297,Katoenen draagtas gekleurd,06,Expeditie,720,3,Expeditie
6,3,2024-07-24 00:00:00,2024-07-09 08:11:10,67053bafda93609bc1305299,Avant-knijper,13,Vullen,6120,0,Soweco


In [334]:
out_df = pd.DataFrame(flat_out).sort_values(by=['order_id', 'sort_order'], ascending=True).convert_dtypes()
out_df['sort_order'] = out_df.groupby(['order_id', 'product_id'])['sort_order'].transform(lambda x: x.rank(method='dense').astype(int) - 1)
out_df = out_df.map(safe_to_datetime)
out_df.head()

,order_id,delivery_date,order_created_at,product_id,material,color,task_title,task_duration,sort_order,workspace,start_at,end_at
3,1,2024-07-13 14:28:51,2024-07-08 14:28:51,67053bafda93609bc1305297,Katoenen draagtas gekleurd,06,Vullen,3600,0,Soweco,2024-07-01 09:00:00,2024-07-01 10:00:00
2,1,2024-07-13 14:28:51,2024-07-08 14:28:51,67053bafda93609bc1305297,Katoenen draagtas gekleurd,06,Tampon drukken (TB),1440,1,T2 1|2|3,2024-07-08 09:00:00,2024-07-08 09:24:00
0,1,2024-07-13 14:28:51,2024-07-08 14:28:51,67053bafda93609bc1305297,Katoenen draagtas gekleurd,06,Inpakken,180,2,Inpak,2024-07-11 09:00:00,2024-07-11 09:03:00
1,1,2024-07-13 14:28:51,2024-07-08 14:28:51,67053bafda93609bc1305297,Katoenen draagtas gekleurd,06,Expeditie,720,3,Expeditie,2024-07-12 09:00:00,2024-07-12 09:12:00
6,3,2024-07-24 00:00:00,2024-07-09 08:11:10,67053bafda93609bc1305299,Avant-knijper,13,Vullen,6120,0,Soweco,2024-07-10 09:00:00,2024-07-10 10:42:00
